In [1]:
import os

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig

#//*** create model output path if needed
output_path = "tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [2]:
from TTS.tts.utils.text import cleaners
dir(cleaners)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_whitespace_re',
 'abbreviations_en',
 'abbreviations_fr',
 'anyascii',
 'basic_cleaners',
 'basic_german_cleaners',
 'basic_turkish_cleaners',
 'chinese_mandarin_cleaners',
 'collapse_whitespace',
 'convert_to_ascii',
 'en_normalize_numbers',
 'english_cleaners',
 'expand_abbreviations',
 'expand_time_english',
 'french_cleaners',
 'lowercase',
 'multilingual_cleaners',
 'no_cleaners',
 'phoneme_cleaners',
 'portuguese_cleaners',
 're',
 'remove_aux_symbols',
 'replace_numbers_to_characters_in_text',
 'replace_symbols',
 'transliteration_cleaners']

In [3]:
output_path = "tts_train_dir"
training_dir = "training"
output_path = os.path.join(os.path.join(os.getcwd(), output_path))
output_path
training_dir = os.path.join(os.path.join(os.getcwd(), training_dir)).replace("\\","/")
print(str(training_dir))
t_path = str(output_path).replace("\\","/")
t_path
print(t_path)

import os
import wave
fulldir = os.path.join(output_path,"wavs")
#//**** Get the sample rate of the first WAV file, assume all files share the same sample_rate
for file_name in os.listdir(fulldir):
    with wave.open(os.path.join(fulldir,file_name), "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        print(file_name,frame_rate)
        break


C:/Users/stonk013/KGO_Projects/tts/ama_model/training
C:/Users/stonk013/KGO_Projects/tts/ama_model/tts_train_dir
ama-01.wav 48000


In [4]:
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples


# dataset config for one of the pre-defined datasets
dataset_config = BaseDatasetConfig(
    #formatter="vctk", meta_file_train="", language="en-us", path=os.path.join(os.path.join(output_path,"wavs"))
    formatter="ljspeech", meta_file_train="metadata.csv", language="en-us", path=output_path
)

# load training samples
train_samples, eval_samples = load_tts_samples(dataset_config, eval_split=False)

 | > Found 12 files in C:\Users\stonk013\KGO_Projects\tts\ama_model\tts_train_dir


In [5]:
import os

from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseAudioConfig
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.tacotron2_config import Tacotron2Config
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.tacotron2 import Tacotron2
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

In [6]:
audio_config = BaseAudioConfig(
    sample_rate=frame_rate,
    do_trim_silence=True,
    trim_db=60.0,
    signal_norm=False,
    mel_fmin=0.0,
    mel_fmax=8000,
    spec_gain=1.0,
    log_func="np.log",
    ref_level_db=20,
    preemphasis=0.0,
)

In [7]:
config = Tacotron2Config(  # This is the config that is saved for the future use
    audio=audio_config,
    batch_size=64,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    r=6,
    gradual_training=[[0, 6, 64], [10000, 4, 32], [50000, 3, 32], [100000, 2, 32]],
    double_decoder_consistency=True,
    epochs=1000,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(training_dir, "phoneme_cache"),
    precompute_num_workers=8,
    print_step=25,
    print_eval=True,
    mixed_precision=False,
    output_path=training_dir,
    datasets=[dataset_config],
)

In [8]:
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
config = GlowTTSConfig(
    audio=audio_config,
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(training_dir, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    save_step=1000,
    
)

In [9]:
output_path

'C:\\Users\\stonk013\\KGO_Projects\\tts\\ama_model\\tts_train_dir'

In [10]:
# init audio processor
ap = AudioProcessor(**config.audio.to_dict())

# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# If characters are not defined in the config, default characters are passed to the config
tokenizer, config = TTSTokenizer.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:48000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60.0
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Setting up Audio Processor...
 | > sample_rate:48000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5


In [11]:
output_path

'C:\\Users\\stonk013\\KGO_Projects\\tts\\ama_model\\tts_train_dir'

In [12]:
#model = Tacotron2(config, ap, tokenizer, speaker_manager=None)

In [13]:
from TTS.tts.models.glow_tts import GlowTTS
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

In [14]:
# INITIALIZE THE MODEL
# Models take a config object and a speaker manager as input
# Config defines the details of the model like the number of layers, the size of the embedding, etc.
# Speaker manager is used by multi-speaker models.


# init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)
trainer.fit()

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Num. of CPUs: 8
 | > Num. of Torch Threads: 4
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=C:\Users\stonk013\KGO_Projects\tts\ama_model\tts_train_dir\run-August-10-2023_09+04PM-0000000

 > Model has 28610257 parameters

 > EPOCH: 0/100
 --> C:\Users\stonk013\KGO_Projects\tts\ama_model\tts_train_dir\run-August-10-2023_09+04PM-0000000

 > TRAINING (2023-08-10 21:05:00) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
| > Number of instances : 12
 | > Preprocessing samples
 | > Max text length: 908
 | > Min text length: 106
 | > Avg text length: 392.8181818181818
 | 
 | > Max audio length: 4811245.0
 | > Min audio length: 645483.0
 | > Avg audio length: 2134389.1818181816
 | > Num. instances discarded samples: 1
 | > Batch group size: 0.
Self.OutPath C:\Users\stonk013\KGO_Projects\tts\ama_model\tts_train_dir\run-August-10-2023_09+04PM-0000000


Traceback (most recent call last):
  File "C:\Users\stonk013\Anaconda3\envs\tts\lib\site-packages\trainer\trainer.py", line 1806, in fit
    self._fit()
  File "C:\Users\stonk013\Anaconda3\envs\tts\lib\site-packages\trainer\trainer.py", line 1758, in _fit
    self.train_epoch()
  File "C:\Users\stonk013\Anaconda3\envs\tts\lib\site-packages\trainer\trainer.py", line 1485, in train_epoch
    for cur_step, batch in enumerate(self.train_loader):
  File "C:\Users\stonk013\Anaconda3\envs\tts\lib\site-packages\torch\utils\data\dataloader.py", line 633, in __next__
    data = self._next_data()
  File "C:\Users\stonk013\Anaconda3\envs\tts\lib\site-packages\torch\utils\data\dataloader.py", line 1345, in _next_data
    return self._process_data(data)
  File "C:\Users\stonk013\Anaconda3\envs\tts\lib\site-packages\torch\utils\data\dataloader.py", line 1371, in _process_data
    data.reraise()
  File "C:\Users\stonk013\Anaconda3\envs\tts\lib\site-packages\torch\_utils.py", line 643, in reraise
    r

AttributeError: 'tuple' object has no attribute 'tb_frame'